In [5]:
import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [9]:
import tensorflow as tf
import scipy
import warnings
warnings.filterwarnings("ignore")

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply((tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output


In [10]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.5})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0, Step: 0, Loss: 36.6247
Epoch: 0, Step: 10, Loss: 31.1107
Epoch: 0, Step: 20, Loss: 26.0724
Epoch: 0, Step: 30, Loss: 21.9718
Epoch: 0, Step: 40, Loss: 18.6133
Epoch: 0, Step: 50, Loss: 15.9053
Epoch: 0, Step: 60, Loss: 13.7861
Epoch: 0, Step: 70, Loss: 12.7727
Epoch: 0, Step: 80, Loss: 11.8995
Epoch: 0, Step: 90, Loss: 9.06106
Epoch: 0, Step: 100, Loss: 7.72858
Epoch: 0, Step: 110, Loss: 6.77327
Epoch: 0, Step: 120, Loss: 6.01753
Epoch: 0, Step: 130, Loss: 5.37802
Epoch: 0, Step: 140, Loss: 4.85477
Epoch: 0, Step: 150, Loss: 4.45101
Epoch: 0, Step: 160, Loss: 4.03251
Epoch: 0, Step: 170, Loss: 3.67764
Epoch: 0, Step: 180, Loss: 3.47405
Epoch: 0, Step: 190, Loss: 3.52288
Epoch: 0, Step: 200, Loss: 3.01342
Epoch: 0, Step: 210, Loss: 2.92458
Epoch: 0, Step: 220, Loss: 2.63165
Epoch: 0, Step: 230, Loss: 2.50022
Epoch: 0, Step: 240, Loss: 2.38463
Epoch: 0, Step: 250, Loss: 2.28186
Epoch: 0, Step: 260, Loss: 

Epoch: 2, Step: 230, Loss: 0.449413
Epoch: 2, Step: 240, Loss: 0.417682
Epoch: 2, Step: 250, Loss: 0.52143
Epoch: 2, Step: 260, Loss: 1.43837
Epoch: 2, Step: 270, Loss: 1.69389
Epoch: 2, Step: 280, Loss: 0.440034
Epoch: 2, Step: 290, Loss: 0.486933
Epoch: 2, Step: 300, Loss: 0.358259
Epoch: 2, Step: 310, Loss: 0.35415
Epoch: 2, Step: 320, Loss: 0.351953
Epoch: 2, Step: 330, Loss: 0.350333
Epoch: 2, Step: 340, Loss: 0.369661
Epoch: 2, Step: 350, Loss: 0.348857
Epoch: 2, Step: 360, Loss: 0.322166
Epoch: 2, Step: 370, Loss: 0.441025
Epoch: 2, Step: 380, Loss: 0.731755
Epoch: 2, Step: 390, Loss: 0.447653
Epoch: 2, Step: 400, Loss: 0.39245
Epoch: 2, Step: 410, Loss: 0.297223
Epoch: 2, Step: 420, Loss: 0.29168
Epoch: 2, Step: 430, Loss: 0.285231
Epoch: 2, Step: 440, Loss: 0.278894
Epoch: 2, Step: 450, Loss: 0.335828
Epoch: 2, Step: 460, Loss: 5.61556
Epoch: 2, Step: 470, Loss: 0.448395
Epoch: 2, Step: 480, Loss: 0.261897
Epoch: 2, Step: 490, Loss: 0.25116
Epoch: 2, Step: 500, Loss: 0.251073


Epoch: 4, Step: 530, Loss: 0.14501
Epoch: 4, Step: 540, Loss: 0.085331
Epoch: 4, Step: 550, Loss: 0.0961847
Epoch: 4, Step: 560, Loss: 0.201424
Epoch: 4, Step: 570, Loss: 0.477005
Epoch: 4, Step: 580, Loss: 0.304165
Epoch: 4, Step: 590, Loss: 0.0892685
Epoch: 4, Step: 600, Loss: 0.0834278
Epoch: 4, Step: 610, Loss: 0.0828766
Epoch: 4, Step: 620, Loss: 0.0819526
Epoch: 4, Step: 630, Loss: 0.0808744
Epoch: 4, Step: 640, Loss: 0.547742
Epoch: 4, Step: 650, Loss: 4.9729
Epoch: 4, Step: 660, Loss: 0.109469
Epoch: 4, Step: 670, Loss: 0.0771629
Epoch: 4, Step: 680, Loss: 0.0728334
Epoch: 4, Step: 690, Loss: 0.0784374
Epoch: 4, Step: 700, Loss: 0.0726191
Epoch: 4, Step: 710, Loss: 0.0697292
Epoch: 4, Step: 720, Loss: 0.0622041
Epoch: 4, Step: 730, Loss: 0.0617844
Epoch: 4, Step: 740, Loss: 0.0608753
Epoch: 4, Step: 750, Loss: 0.0595988
Epoch: 4, Step: 760, Loss: 0.0614007
Epoch: 4, Step: 770, Loss: 0.0950736
Epoch: 4, Step: 780, Loss: 0.096245
Epoch: 4, Step: 790, Loss: 0.0705003
Epoch: 4, Ste

Epoch: 6, Step: 810, Loss: 0.0339144
Epoch: 6, Step: 820, Loss: 0.0366273
Epoch: 6, Step: 830, Loss: 1.55502
Epoch: 6, Step: 840, Loss: 4.0237
Epoch: 6, Step: 850, Loss: 0.0364185
Epoch: 6, Step: 860, Loss: 0.0256504
Epoch: 6, Step: 870, Loss: 0.0251285
Epoch: 6, Step: 880, Loss: 0.0290215
Epoch: 6, Step: 890, Loss: 0.0256704
Epoch: 6, Step: 900, Loss: 0.0262508
Epoch: 6, Step: 910, Loss: 0.0241214
Epoch: 6, Step: 920, Loss: 0.0238713
Epoch: 6, Step: 930, Loss: 0.0241576
Epoch: 6, Step: 940, Loss: 0.0208448
Epoch: 6, Step: 950, Loss: 0.0257138
Epoch: 6, Step: 960, Loss: 0.0604984
Epoch: 6, Step: 970, Loss: 0.0594859
Epoch: 6, Step: 980, Loss: 0.0277411
Epoch: 6, Step: 990, Loss: 0.0192833
Epoch: 6, Step: 1000, Loss: 0.0195815
Epoch: 6, Step: 1010, Loss: 0.0196517
Epoch: 6, Step: 1020, Loss: 0.0186013
Epoch: 6, Step: 1030, Loss: 0.0191515
Epoch: 6, Step: 1040, Loss: 0.0185368
Epoch: 6, Step: 1050, Loss: 0.0175729
Model saved in file: ./save\model.ckpt
Epoch: 7, Step: 700, Loss: 0.018525

Epoch: 8, Step: 1110, Loss: 0.00832673
Epoch: 8, Step: 1120, Loss: 0.0090587
Epoch: 8, Step: 1130, Loss: 0.00717053
Epoch: 8, Step: 1140, Loss: 0.0182604
Epoch: 8, Step: 1150, Loss: 0.0575584
Epoch: 8, Step: 1160, Loss: 0.0499677
Epoch: 8, Step: 1170, Loss: 0.0072949
Epoch: 8, Step: 1180, Loss: 0.00713995
Epoch: 8, Step: 1190, Loss: 0.00766531
Epoch: 8, Step: 1200, Loss: 0.00769843
Epoch: 8, Step: 1210, Loss: 0.00725166
Epoch: 8, Step: 1220, Loss: 0.0101064
Epoch: 8, Step: 1230, Loss: 0.00629498
Epoch: 8, Step: 1240, Loss: 0.00734485
Epoch: 8, Step: 1250, Loss: 0.0104089
Model saved in file: ./save\model.ckpt
Epoch: 9, Step: 900, Loss: 0.007998
Epoch: 9, Step: 910, Loss: 0.00934799
Epoch: 9, Step: 920, Loss: 0.0074674
Epoch: 9, Step: 930, Loss: 0.00714448
Epoch: 9, Step: 940, Loss: 0.00740369
Epoch: 9, Step: 950, Loss: 0.00813779
Epoch: 9, Step: 960, Loss: 0.0422124
Epoch: 9, Step: 970, Loss: 0.0383523
Epoch: 9, Step: 980, Loss: 4.99991
Epoch: 9, Step: 990, Loss: 0.461507
Epoch: 9, Ste

...

Epoch: 21, Step: 2310, Loss: 0.0213026
Epoch: 21, Step: 2320, Loss: 0.0222491
Epoch: 21, Step: 2330, Loss: 0.023122
Epoch: 21, Step: 2340, Loss: 0.0234743
Epoch: 21, Step: 2350, Loss: 0.0228688
Epoch: 21, Step: 2360, Loss: 0.0272285
Epoch: 21, Step: 2370, Loss: 0.0330679
Epoch: 21, Step: 2380, Loss: 0.0408726
Epoch: 21, Step: 2390, Loss: 0.00868936
Epoch: 21, Step: 2400, Loss: 0.0194865
Epoch: 21, Step: 2410, Loss: 0.0822627
Epoch: 21, Step: 2420, Loss: 0.0237167
Epoch: 21, Step: 2430, Loss: 0.321274
Epoch: 21, Step: 2440, Loss: 0.604604
Epoch: 21, Step: 2450, Loss: 0.229476
Epoch: 21, Step: 2460, Loss: 0.0673618
Epoch: 21, Step: 2470, Loss: 0.0362181
Epoch: 21, Step: 2480, Loss: 0.01706
Epoch: 21, Step: 2490, Loss: 0.125838
Epoch: 21, Step: 2500, Loss: 0.948475
Epoch: 21, Step: 2510, Loss: 1.43941
Epoch: 21, Step: 2520, Loss: 0.120001
Epoch: 21, Step: 2530, Loss: 0.119187
Epoch: 21, Step: 2540, Loss: 0.00133029
Epoch: 21, Step: 2550, Loss: 0.00560822
Model saved in file: ./save\model.

Epoch: 23, Step: 2540, Loss: 0.0154872
Epoch: 23, Step: 2550, Loss: 0.0278404
Epoch: 23, Step: 2560, Loss: 0.0240585
Epoch: 23, Step: 2570, Loss: 0.0339476
Epoch: 23, Step: 2580, Loss: 0.00486772
Epoch: 23, Step: 2590, Loss: 0.0290854
Epoch: 23, Step: 2600, Loss: 0.0946102
Epoch: 23, Step: 2610, Loss: 0.0146024
Epoch: 23, Step: 2620, Loss: 0.452482
Epoch: 23, Step: 2630, Loss: 0.615761
Epoch: 23, Step: 2640, Loss: 0.208645
Epoch: 23, Step: 2650, Loss: 0.0490764
Epoch: 23, Step: 2660, Loss: 0.0400459
Epoch: 23, Step: 2670, Loss: 0.0148125
Epoch: 23, Step: 2680, Loss: 0.141547
Epoch: 23, Step: 2690, Loss: 1.14785
Epoch: 23, Step: 2700, Loss: 1.16898
Epoch: 23, Step: 2710, Loss: 0.0534121
Epoch: 23, Step: 2720, Loss: 0.11127
Epoch: 23, Step: 2730, Loss: 0.00246345
Epoch: 23, Step: 2740, Loss: 0.00371471
Epoch: 23, Step: 2750, Loss: 0.0097739
Model saved in file: ./save\model.ckpt
Epoch: 24, Step: 2400, Loss: 0.0168133
Epoch: 24, Step: 2410, Loss: 0.0470461
Epoch: 24, Step: 2420, Loss: 0.0

Epoch: 25, Step: 2810, Loss: 0.538156
Epoch: 25, Step: 2820, Loss: 0.536761
Epoch: 25, Step: 2830, Loss: 0.14864
Epoch: 25, Step: 2840, Loss: 0.0337273
Epoch: 25, Step: 2850, Loss: 0.0213924
Epoch: 25, Step: 2860, Loss: 0.0414288
Epoch: 25, Step: 2870, Loss: 0.293552
Epoch: 25, Step: 2880, Loss: 1.44688
Epoch: 25, Step: 2890, Loss: 0.898855
Epoch: 25, Step: 2900, Loss: 0.0659008
Epoch: 25, Step: 2910, Loss: 0.0541608
Epoch: 25, Step: 2920, Loss: 0.00434417
Epoch: 25, Step: 2930, Loss: 0.00389084
Epoch: 25, Step: 2940, Loss: 0.00947335
Epoch: 25, Step: 2950, Loss: 0.0259116
Model saved in file: ./save\model.ckpt
Epoch: 26, Step: 2600, Loss: 0.0606205
Epoch: 26, Step: 2610, Loss: 0.00213099
Epoch: 26, Step: 2620, Loss: 0.036514
Epoch: 26, Step: 2630, Loss: 0.138803
Epoch: 26, Step: 2640, Loss: 0.383614
Epoch: 26, Step: 2650, Loss: 0.22824
Epoch: 26, Step: 2660, Loss: 0.0186055
Epoch: 26, Step: 2670, Loss: 0.0132716
Epoch: 26, Step: 2680, Loss: 0.0146124
Epoch: 26, Step: 2690, Loss: 0.015

Epoch: 27, Step: 3080, Loss: 0.595773
Epoch: 27, Step: 3090, Loss: 0.112391
Epoch: 27, Step: 3100, Loss: 0.0238477
Epoch: 27, Step: 3110, Loss: 0.0056166
Epoch: 27, Step: 3120, Loss: 0.000741181
Epoch: 27, Step: 3130, Loss: 0.0101184
Epoch: 27, Step: 3140, Loss: 0.027207
Epoch: 27, Step: 3150, Loss: 0.0506152
Model saved in file: ./save\model.ckpt
Epoch: 28, Step: 2800, Loss: 0.00208423
Epoch: 28, Step: 2810, Loss: 0.0755338
Epoch: 28, Step: 2820, Loss: 0.154008
Epoch: 28, Step: 2830, Loss: 0.339996
Epoch: 28, Step: 2840, Loss: 0.208142
Epoch: 28, Step: 2850, Loss: 0.0114612
Epoch: 28, Step: 2860, Loss: 0.0083705
Epoch: 28, Step: 2870, Loss: 0.00849545
Epoch: 28, Step: 2880, Loss: 0.00765895
Epoch: 28, Step: 2890, Loss: 0.013423
Epoch: 28, Step: 2900, Loss: 2.16661
Epoch: 28, Step: 2910, Loss: 3.38541
Epoch: 28, Step: 2920, Loss: 0.0133398
Epoch: 28, Step: 2930, Loss: 0.00434012
Epoch: 28, Step: 2940, Loss: 0.00415988
Epoch: 28, Step: 2950, Loss: 0.0111367
Epoch: 28, Step: 2960, Loss: 

Epoch: 29, Step: 3310, Loss: 0.00346186
Epoch: 29, Step: 3320, Loss: 0.0123203
Epoch: 29, Step: 3330, Loss: 0.040035
Epoch: 29, Step: 3340, Loss: 0.0485
Epoch: 29, Step: 3350, Loss: 0.00542538
Model saved in file: ./save\model.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser


In [1]:
#pip3 install opencv-python

import tensorflow as tf
import scipy.misc
import model
import cv2
from subprocess import call
import math

import os
import numpy as np
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


C:\Users\CHALLA VENKAT RAO\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Restoring parameters from save/model.ckpt
Starting frameofvideo:36325


C:\Users\CHALLA VENKAT RAO\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\CHALLA VENKAT RAO\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Steering angle: 45.350389956660734 (pred)	-10.79 (actual)
Steering angle: 41.55346730648314 (pred)	-10.08 (actual)
Steering angle: 39.535699652067144 (pred)	-9.380000000000003 (actual)
Steering angle: 31.624489611927302 (pred)	-7.559999999999999 (actual)
Steering angle: 35.189332679391626 (pred)	-5.949999999999999 (actual)
Steering angle: 42.36086738336493 (pred)	-3.23 (actual)
Steering angle: 42.21900093920918 (pred)	-0.81 (actual)
Steering angle: 43.9297310998656 (pred)	1.21 (actual)
Steering angle: 38.2286302562639 (pred)	3.4300000000000006 (actual)
Steering angle: 30.664670448622093 (pred)	4.44 (actual)
Steering angle: 30.355467784895616 (pred)	5.340000000000001 (actual)
Steering angle: 27.290003753313023 (pred)	5.75 (actual)
Steering angle: 28.930324908910357 (pred)	7.16 (actual)
Steering angle: 25.087122604464046 (pred)	8.27 (actual)
Steering angle: 34.92855947197512 (pred)	9.48 (actual)
Steering angle: 35.43160290434656 (pred)	0.0 (actual)
Steering angle: 34.96052817238537 (pred

Steering angle: 28.654378436252674 (pred)	47.7 (actual)
Steering angle: 25.73433425474153 (pred)	47.7 (actual)
Steering angle: 26.497314196356143 (pred)	47.7 (actual)
Steering angle: 26.010316585892358 (pred)	47.7 (actual)
Steering angle: 37.566248755839815 (pred)	47.8 (actual)
Steering angle: 41.4551023371512 (pred)	47.7 (actual)
Steering angle: 48.254541995520555 (pred)	47.7 (actual)
Steering angle: 37.28325352795868 (pred)	47.7 (actual)
Steering angle: 46.43089514750549 (pred)	47.7 (actual)
Steering angle: 46.4039944474594 (pred)	47.7 (actual)
Steering angle: 48.76651590023179 (pred)	47.7 (actual)
Steering angle: 43.31044467800971 (pred)	47.7 (actual)
Steering angle: 45.97437896376018 (pred)	47.8 (actual)
Steering angle: 40.35940680559234 (pred)	47.8 (actual)
Steering angle: 34.33827403333511 (pred)	48.0 (actual)
Steering angle: 20.88480773647868 (pred)	0.0 (actual)
Steering angle: 26.072688165845623 (pred)	48.5 (actual)
Steering angle: 22.902925438089643 (pred)	48.71000000000001 (a

...

Steering angle: 27.542309233706 (pred)	13.82 (actual)
Steering angle: 30.6538719195443 (pred)	13.610000000000001 (actual)
Steering angle: 30.59091465086971 (pred)	13.41 (actual)
Steering angle: 26.10148595093369 (pred)	13.41 (actual)
Steering angle: 29.909569130215086 (pred)	13.41 (actual)
Steering angle: 32.23157731592555 (pred)	13.41 (actual)
Steering angle: 27.112712533027 (pred)	13.41 (actual)
Steering angle: 29.52156657403417 (pred)	13.41 (actual)
Steering angle: 26.340168619027075 (pred)	13.41 (actual)
Steering angle: 29.05575108773389 (pred)	13.41 (actual)
Steering angle: 28.617672999653024 (pred)	13.41 (actual)
Steering angle: 24.59696773905642 (pred)	13.41 (actual)
Steering angle: 22.356593974305955 (pred)	13.41 (actual)
Steering angle: 26.73870844955003 (pred)	13.41 (actual)
Steering angle: 24.837795086549193 (pred)	13.41 (actual)
Steering angle: 32.63901517538973 (pred)	13.41 (actual)
Steering angle: 29.54187955662579 (pred)	13.41 (actual)
Steering angle: 23.323917807961717 

Steering angle: 25.758875124429387 (pred)	3.9300000000000006 (actual)
Steering angle: 25.423714326762276 (pred)	3.9300000000000006 (actual)
Steering angle: 28.524008908017347 (pred)	4.03 (actual)
Steering angle: 27.616073569194846 (pred)	4.03 (actual)
Steering angle: 28.758040217285956 (pred)	4.03 (actual)
Steering angle: 28.1969196864019 (pred)	3.9300000000000006 (actual)
Steering angle: 31.55366738042344 (pred)	3.829999999999999 (actual)
Steering angle: 29.706149021259183 (pred)	3.63 (actual)
Steering angle: 32.627400438706125 (pred)	3.63 (actual)
Steering angle: 31.365779121640145 (pred)	3.63 (actual)
Steering angle: 32.91045713828979 (pred)	3.53 (actual)
Steering angle: 28.443820779614107 (pred)	3.23 (actual)
Steering angle: 29.837124728783348 (pred)	3.03 (actual)
Steering angle: 27.510847674841628 (pred)	2.92 (actual)
Steering angle: 28.696285061905286 (pred)	2.92 (actual)
Steering angle: 26.630120394577872 (pred)	2.7200000000000006 (actual)
Steering angle: 28.82434769374844 (pred

Steering angle: 25.881210642653485 (pred)	12.4 (actual)
Steering angle: 26.136500915713757 (pred)	12.61 (actual)
Steering angle: 26.030472474133067 (pred)	13.01 (actual)
Steering angle: 28.381923897808157 (pred)	13.31 (actual)
Steering angle: 25.94545881708121 (pred)	13.610000000000001 (actual)
Steering angle: 27.290235979744804 (pred)	13.92 (actual)
Steering angle: 28.53215561614988 (pred)	13.92 (actual)
Steering angle: 26.577111296429404 (pred)	13.92 (actual)
Steering angle: 24.191273285377882 (pred)	14.02 (actual)
Steering angle: 24.199134833112566 (pred)	14.02 (actual)
Steering angle: 25.59710038639034 (pred)	14.02 (actual)
Steering angle: 25.82059100622233 (pred)	14.02 (actual)
Steering angle: 29.90233254478946 (pred)	14.12 (actual)
Steering angle: 29.422969378270455 (pred)	14.12 (actual)
Steering angle: 29.53649053737065 (pred)	14.12 (actual)
Steering angle: 23.976660203919618 (pred)	14.12 (actual)
Steering angle: 26.984272533232204 (pred)	14.12 (actual)
Steering angle: 26.145419

Steering angle: 35.70502220701134 (pred)	5.949999999999999 (actual)
Steering angle: 29.82104646347714 (pred)	5.75 (actual)
Steering angle: 25.878727868890184 (pred)	5.340000000000001 (actual)
Steering angle: 31.556857078765837 (pred)	5.139999999999999 (actual)
Steering angle: 38.13641245718497 (pred)	4.94 (actual)
Steering angle: 38.03720054439595 (pred)	4.64 (actual)
Steering angle: 38.31078377160081 (pred)	4.24 (actual)
Steering angle: 38.60610066074465 (pred)	4.24 (actual)
Steering angle: 34.31128795592439 (pred)	3.829999999999999 (actual)
Steering angle: 36.326074232767674 (pred)	3.63 (actual)
Steering angle: 31.561983135737933 (pred)	3.13 (actual)
Steering angle: 33.0878883774538 (pred)	2.62 (actual)
Steering angle: 33.91338504054154 (pred)	2.22 (actual)
Steering angle: 39.95522694636225 (pred)	1.6100000000000003 (actual)
Steering angle: 35.82961510275613 (pred)	1.6100000000000003 (actual)
Steering angle: 39.005246670556446 (pred)	1.6100000000000003 (actual)
Steering angle: 37.791

IndexError: list index out of range